# **Make Pretrained-Model FaceNet !**


## 0.미션


### (1) 미션1
여러분은 노트북에서 얼굴 인식 파일을 실행시키기 위해 사전 학습 모델을 만들어야 합니다.

그 전에 가지고 있는 데이터셋을 **학습에 적합한 형태**로 만들어야 합니다.

- 1) 데이터셋을 불러옵니다.
    - 데이터셋은 2가지입니다. 본인의 얼굴 이미지 파일, 다른 사람의 얼굴 이미지 파일.
- 2) 데이터셋을 전처리합니다.
    - Keras에는 **실제로 존재하는 이미지 데이터를 처리**해주는 함수가 있습니다.

### (2) 미션2
데이터셋을 **학습에 적합한 형태**로 만들었다면, **FaceNet 모델로 Transfer Learning**을 수행합니다.

- 1) FaceNet 모델 구조를 생성합니다.
    - [FaceNet 논문 링크](https://arxiv.org/abs/1503.03832)
    - FaceNet의 Input은 (160, 160) 사이즈의 이미지입니다.
- 2) FaceNet 모델 구조 + 구조 추가
    - FaceNet의 Output은 128차원의 벡터입니다.
    - 이 과정을 Transfer Learning라고 합니다.

### (3) 미션3
학습된 모델로 추론하여 성능 지표를 확인하고 모델을 개선시키세요.

그 후, 모델의 구조와 가중치를 **반드시 저장**하여 여러분의 노트북에 옮기세요.

- 1) 다양한 모델을 사용해보세요.
    - 모델에 정해진 정답은 없습니다.
    - 성능 지표에서 무엇이 중요한지 깊게 생각하세요.
    - 사전 학습된 FaceNet 모델을 사용하셔도 좋고, 아예 독창적으로 여러분만의 모델을 만드셔도 좋습니다!
- 2) 모델을 **반드시 저장**하세요.
    - .keras 형태로 우선 Colab에 저장하세요.
    - Colab에 생성된 .keras 파일을 **로컬에 다운로드** 합니다.

## 1.환경설정

* 세부 요구사항
    - 경로 설정 : 구글콜랩
        * 구글 드라이브 바로 밑에 project4 폴더를 만드세요.
        * 데이터 파일을 복사해 넣습니다.
        * 필요하다고 판단되는 라이브러리를 추가하세요.

### (1) 경로 설정

* 구글 드라이브 연결

In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
path = '/content/drive/MyDrive/project4'

### (2) 라이브러리 설치 및 불러오기

* 라이브러리 로딩

In [46]:
## colab에서 세션 재시작을 요구하는 팝업이 뜨면 재시작 누르세요.
!pip install keras-nightly

## 3.미션1

여러분은 노트북에서 얼굴 인식 파일을 실행시키기 위해 사전 학습 모델을 만들어야 합니다.

그 전에 가지고 있는 데이터셋을 **학습에 적합한 형태**로 만들어야 합니다.

- 1) 데이터셋을 불러옵니다.
    - 데이터셋은 2가지입니다. 본인의 얼굴 이미지 파일, 다른 사람의 얼굴 이미지 파일.
- 2) 데이터셋을 전처리합니다.
    - Keras에는 **실제로 존재하는 이미지 데이터를 처리**해주는 함수가 있습니다.

### (1) 데이터셋 불러오기

* **세부 요구사항**
    - 데이터셋을 불러옵니다.
        - 데이터셋은 두 개의 압축 파일이어야 합니다.
            1. lfw-deepfunneled.zip : Labeled Faces in the Wild 데이터셋
            2. 여러분의 얼굴 이미지 데이터셋
                - 여러분의 얼굴 이미지가 담긴 **압축 파일**을 **Google Drive에 업로드** 하기를 권장합니다.
                    - 이미지 파일 하나하나 업로드 하면 시간이 오래 걸립니다.
    - 데이터셋 압축 파일을 **Colab에 폴더를 생성한 후 해제**하세요.
        - 데이터셋 폴더를 **본인 얼굴 폴더, LFW 폴더로 나누어** 생성하는 것을 권장합니다.
        - 만일 두 압축 파일을 하나의 폴더에 모두 해제하면 전처리가 더 까다로워질 것입니다.
    - 예시 코드에서 사용한 라이브러리
        - os, zipfile

#### 1) 본인 얼굴 이미지 데이터셋 불러오기

In [47]:
import keras
import zipfile
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import Image
import numpy as np
from keras.utils import load_img, img_to_array

import numpy as np
import matplotlib.pyplot as plt

In [48]:
import zipfile
import numpy as np
from PIL import Image

# 압축 파일 경로
zip_path = '/content/drive/MyDrive/project4/my_face_exam.zip'

# 이미지 데이터를 저장할 리스트
my_image_data = []

# ZIP 파일 열기
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # ZIP 파일 내 모든 파일 탐색
    for file_name in zip_ref.namelist():
        # __MACOSX 폴더 및 ._로 시작하는 파일 제외
        if file_name.startswith('__MACOSX') or file_name.startswith('._'):
            continue

        if file_name.endswith(('.png', '.jpg', '.jpeg')):  # 이미지 파일 필터링
            # 이미지 파일을 메모리로 읽어오기
            with zip_ref.open(file_name) as file:
                try:
                    # 이미지를 PIL 형식으로 열고 배열로 변환
                    img = Image.open(file)
                    img = img.resize((160, 160))  # 이미지 크기를 (160, 160)으로 조정
                    img_array = np.array(img)  # 이미지를 배열로 변환
                    my_image_data.append(img_array)
                except Exception as e:
                    print(f"오류 발생: {file_name}, {e}")

# 리스트를 Numpy 배열로 변환
my_image_data = np.array(my_image_data)

# 결과 확인
print(f"불러온 이미지 개수: {len(my_image_data)}")
print(f"첫 번째 이미지 크기: {my_image_data[0].shape}")


불러온 이미지 개수: 3000
첫 번째 이미지 크기: (160, 160, 3)


#### 2) 다른 얼굴 이미지 데이터셋 불러오기

In [49]:
import zipfile
import numpy as np
from PIL import Image
from io import BytesIO

In [50]:
import zipfile
import numpy as np
from PIL import Image
from io import BytesIO

# 압축 파일 경로
zip_path = '/content/drive/MyDrive/project4/Datasets/Keras/lfw-deepfunneled.zip'

# 이미지 데이터를 저장할 리스트
face_image = []

# ZIP 파일 열기
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # ZIP 파일 내 모든 파일 탐색
    for file_name in zip_ref.namelist():
        # __MACOSX 폴더 및 ._로 시작하는 파일 제외
        if file_name.startswith('__MACOSX') or file_name.startswith('._'):
            continue

        if file_name.endswith(('.png', '.jpg', '.jpeg')):  # 이미지 파일 필터링
            # 이미지 파일을 메모리로 읽어오기
            with zip_ref.open(file_name) as file:
                try:
                    # 이미지를 PIL 형식으로 열고 배열로 변환
                    img = Image.open(file)
                    img = img.resize((160, 160))  # 이미지 크기를 (160, 160)으로 조정
                    img_array = np.array(img)  # 이미지를 배열로 변환
                    face_image.append(img_array)
                except Exception as e:
                    print(f"오류 발생: {file_name}, {e}")

# 리스트를 Numpy 배열로 변환
face_image = np.array(face_image)

# 결과 확인
print(f"불러온 이미지 개수: {len(face_image)}")
print(f"첫 번째 이미지 크기: {face_image[0].shape}")


불러온 이미지 개수: 13233
첫 번째 이미지 크기: (160, 160, 3)


### (2) 데이터셋 전처리
* **세부 요구사항**
    - 데이터셋을 전처리 합니다.
        - Training set, Validation set, Test set으로 데이터셋을 나누어 주세요.
            - 학습 과정에서 Training set, Validation set을 사용해야 합니다.
            - 추론 과정에서 Test set을 사용해야 합니다.
        - Keras의 **특정 함수**가 실제 존재하는 이미지 파일에 대한 전처리를 쉽게 도와줍니다.
        - **특정 함수**에서 요구하는 폴더 구조가 있습니다. **특정 함수**를 사용한다면 이에 맞춰서 폴더를 생성해야 합니다.
        - 각 데이터셋에 스케일링도 적용하세요.
    - 예시 코드에서 사용한 라이브러리
        - glob, random, shutil, numpy, keras

#### 1) 데이터셋 분할

In [51]:
import numpy as np
from sklearn.model_selection import train_test_split

# 나의 얼굴 데이터셋: my_image_data (예를 들어, 클래스 0)
# 타인의 얼굴 데이터셋: face_image (예를 들어, 클래스 1)

# 1. 각 데이터셋을 개별적으로 Training/Test 세트로 나누기
train_my_face, test_my_face = train_test_split(my_image_data, test_size=0.2, random_state=42)
train_other_face, test_other_face = train_test_split(face_image, test_size=0.2, random_state=42)

# 2. 각 데이터셋에 클래스 레이블 추가 (0 = 내 얼굴, 1 = 타인 얼굴)
train_my_face_labels = np.zeros(len(train_my_face))  # 내 얼굴 레이블 0
test_my_face_labels = np.zeros(len(test_my_face))

train_other_face_labels = np.ones(len(train_other_face))  # 타인 얼굴 레이블 1
test_other_face_labels = np.ones(len(test_other_face))

# 3. 나의 얼굴 데이터셋과 타인의 얼굴 데이터셋을 하나의 Training/Test 세트로 합치기
train_x = np.concatenate([train_my_face, train_other_face], axis=0)
train_y = np.concatenate([train_my_face_labels, train_other_face_labels], axis=0)

test_x = np.concatenate([test_my_face, test_other_face], axis=0)
test_y = np.concatenate([test_my_face_labels, test_other_face_labels], axis=0)

# 결과 확인
print(f"Training set 크기: {train_x.shape}, Training labels 크기: {train_y.shape}")
print(f"Test set 크기: {test_x.shape}, Test labels 크기: {test_y.shape}")


Training set 크기: (12986, 160, 160, 3), Training labels 크기: (12986,)
Test set 크기: (3247, 160, 160, 3), Test labels 크기: (3247,)


#### 2) **특정 함수** 사용

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:

# ImageDataGenerator 설정 (스케일링 없이 사용)
train_datagen = ImageDataGenerator(rescale=1./255)  # 스케일링을 통해 0~1 범위로 정규화
val_datagen = ImageDataGenerator(rescale=1./255)    # 검증용 데이터도 동일하게 스케일링 적용

# 분할된 train_x, train_y로부터 학습 데이터 생성
train_gen = train_datagen.flow(
    train_x,
    train_y,
    batch_size=32,
    shuffle=True
)

# 분할된 test_x, test_y로부터 검증 데이터 생성
val_gen = val_datagen.flow(
    test_x,
    test_y,
    batch_size=32,
    shuffle=False
)


NameError: name 'train_x' is not defined

#### 3) 스케일링

## 4.미션2

데이터셋을 **학습에 적합한 형태**로 만들었다면, **FaceNet 모델로 Transfer Learning**을 수행합니다.

- 1) FaceNet 모델 구조를 생성합니다.
    - [FaceNet 논문 링크](https://arxiv.org/abs/1503.03832)
    - FaceNet의 Input은 (160, 160) 사이즈의 이미지입니다.
- 2) FaceNet 모델 구조 + 구조 추가
    - FaceNet의 Output은 128차원의 벡터입니다.
    - 이 과정을 Transfer Learning라고 합니다.

### (1) FaceNet 구조 생성

* **세부 요구사항**
    - FaceNet의 구조를 생성합니다.
        - FaceNet 구조 생성에 필요한 함수를 만듭니다.
    - FaceNet의 구조에 **잘 학습된 가중치**를 부여합니다.
        - FaceNet 원본 가중치 파일을 공유하였습니다.
    - (선택사항) FaceNet 모델의 가중치 업데이트를 방지합니다.
    - 예시 코드에서 사용한 라이브러리
        - numpy, functools, keras

#### 1) 모델 구조 생성

In [3]:
import numpy as np
from functools import partial


import keras
from keras.models import Model

from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Activation
from keras.layers import BatchNormalization, Dropout, GlobalAveragePooling2D
from keras.layers import Lambda, Concatenate, add

from keras import backend as K
from keras.saving import register_keras_serializable

In [4]:
@register_keras_serializable()
def scaling(x, scale):
    return x * scale

@register_keras_serializable()
def conv2d_bn(x,
              filters,
              kernel_size,
              strides=1,
              padding='same',
              activation='relu',
              use_bias=False,
              name=None):
    x = Conv2D(filters,
               kernel_size,
               strides=strides,
               padding=padding,
               use_bias=use_bias,
               name=name)(x)
    if not use_bias:
        bn_axis = 1 if K.image_data_format() == 'channels_first' else 3
        bn_name = _generate_layer_name('BatchNorm', prefix=name)
        x = BatchNormalization(axis=bn_axis, momentum=0.995, epsilon=0.001,
                               scale=False, name=bn_name)(x)
    if activation is not None:
        ac_name = _generate_layer_name('Activation', prefix=name)
        x = Activation(activation, name=ac_name)(x)
    return x

@register_keras_serializable()
def _generate_layer_name(name, branch_idx=None, prefix=None):
    if prefix is None:
        return None
    if branch_idx is None:
        return '_'.join((prefix, name))
    return '_'.join((prefix, 'Branch', str(branch_idx), name))

@register_keras_serializable()
def _inception_resnet_block(x, scale, block_type, block_idx, activation='relu'):
    channel_axis = 1 if K.image_data_format() == 'channels_first' else 3
    if block_idx is None:
        prefix = None
    else:
        prefix = '_'.join((block_type, str(block_idx)))
    name_fmt = partial(_generate_layer_name, prefix=prefix)

    if block_type == 'Block35':
        branch_0 = conv2d_bn(x, 32, 1, name=name_fmt('Conv2d_1x1', 0))
        branch_1 = conv2d_bn(x, 32, 1, name=name_fmt('Conv2d_0a_1x1', 1))
        branch_1 = conv2d_bn(branch_1, 32, 3, name=name_fmt('Conv2d_0b_3x3', 1))
        branch_2 = conv2d_bn(x, 32, 1, name=name_fmt('Conv2d_0a_1x1', 2))
        branch_2 = conv2d_bn(branch_2, 32, 3, name=name_fmt('Conv2d_0b_3x3', 2))
        branch_2 = conv2d_bn(branch_2, 32, 3, name=name_fmt('Conv2d_0c_3x3', 2))
        branches = [branch_0, branch_1, branch_2]
    elif block_type == 'Block17':
        branch_0 = conv2d_bn(x, 128, 1, name=name_fmt('Conv2d_1x1', 0))
        branch_1 = conv2d_bn(x, 128, 1, name=name_fmt('Conv2d_0a_1x1', 1))
        branch_1 = conv2d_bn(branch_1, 128, [1, 7], name=name_fmt('Conv2d_0b_1x7', 1))
        branch_1 = conv2d_bn(branch_1, 128, [7, 1], name=name_fmt('Conv2d_0c_7x1', 1))
        branches = [branch_0, branch_1]
    elif block_type == 'Block8':
        branch_0 = conv2d_bn(x, 192, 1, name=name_fmt('Conv2d_1x1', 0))
        branch_1 = conv2d_bn(x, 192, 1, name=name_fmt('Conv2d_0a_1x1', 1))
        branch_1 = conv2d_bn(branch_1, 192, [1, 3], name=name_fmt('Conv2d_0b_1x3', 1))
        branch_1 = conv2d_bn(branch_1, 192, [3, 1], name=name_fmt('Conv2d_0c_3x1', 1))
        branches = [branch_0, branch_1]
    else:
        raise ValueError('Unknown Inception-ResNet block type. '
                         'Expects "Block35", "Block17" or "Block8", '
                         'but got: ' + str(block_type))

    mixed = Concatenate(axis=channel_axis, name=name_fmt('Concatenate'))(branches)
    up = conv2d_bn(mixed,
                #    K.int_shape(x)[channel_axis],
                   x.shape[channel_axis],
                   1,
                   activation=None,
                   use_bias=True,
                   name=name_fmt('Conv2d_1x1'))
    up = Lambda(scaling,
                # output_shape=K.int_shape(up)[1:],
                output_shape=up.shape[1:],
                arguments={'scale': scale})(up)
    x = add([x, up])
    if activation is not None:
        x = Activation(activation, name=name_fmt('Activation'))(x)
    return x

In [5]:
@register_keras_serializable()
def InceptionResNetV1(input_shape=(160, 160, 3),
                      classes=128,
                      dropout_keep_prob=0.8,
                      weights_path=None):
    inputs = Input(shape=input_shape)
    x = conv2d_bn(inputs, 32, 3, strides=2, padding='valid', name='Conv2d_1a_3x3')
    x = conv2d_bn(x, 32, 3, padding='valid', name='Conv2d_2a_3x3')
    x = conv2d_bn(x, 64, 3, name='Conv2d_2b_3x3')
    x = MaxPooling2D(3, strides=2, name='MaxPool_3a_3x3')(x)
    x = conv2d_bn(x, 80, 1, padding='valid', name='Conv2d_3b_1x1')
    x = conv2d_bn(x, 192, 3, padding='valid', name='Conv2d_4a_3x3')
    x = conv2d_bn(x, 256, 3, strides=2, padding='valid', name='Conv2d_4b_3x3')

    # 5x Block35 (Inception-ResNet-A block):
    for block_idx in range(1, 6):
        x = _inception_resnet_block(x,
                                    scale=0.17,
                                    block_type='Block35',
                                    block_idx=block_idx)

    # Mixed 6a (Reduction-A block):
    channel_axis = 1 if K.image_data_format() == 'channels_first' else 3
    name_fmt = partial(_generate_layer_name, prefix='Mixed_6a')
    branch_0 = conv2d_bn(x,
                         384,
                         3,
                         strides=2,
                         padding='valid',
                         name=name_fmt('Conv2d_1a_3x3', 0))
    branch_1 = conv2d_bn(x, 192, 1, name=name_fmt('Conv2d_0a_1x1', 1))
    branch_1 = conv2d_bn(branch_1, 192, 3, name=name_fmt('Conv2d_0b_3x3', 1))
    branch_1 = conv2d_bn(branch_1,
                         256,
                         3,
                         strides=2,
                         padding='valid',
                         name=name_fmt('Conv2d_1a_3x3', 1))
    branch_pool = MaxPooling2D(3,
                               strides=2,
                               padding='valid',
                               name=name_fmt('MaxPool_1a_3x3', 2))(x)
    branches = [branch_0, branch_1, branch_pool]
    x = Concatenate(axis=channel_axis, name='Mixed_6a')(branches)

    # 10x Block17 (Inception-ResNet-B block):
    for block_idx in range(1, 11):
        x = _inception_resnet_block(x,
                                    scale=0.1,
                                    block_type='Block17',
                                    block_idx=block_idx)

    # Mixed 7a (Reduction-B block): 8 x 8 x 2080
    name_fmt = partial(_generate_layer_name, prefix='Mixed_7a')
    branch_0 = conv2d_bn(x, 256, 1, name=name_fmt('Conv2d_0a_1x1', 0))
    branch_0 = conv2d_bn(branch_0,
                         384,
                         3,
                         strides=2,
                         padding='valid',
                         name=name_fmt('Conv2d_1a_3x3', 0))
    branch_1 = conv2d_bn(x, 256, 1, name=name_fmt('Conv2d_0a_1x1', 1))
    branch_1 = conv2d_bn(branch_1,
                         256,
                         3,
                         strides=2,
                         padding='valid',
                         name=name_fmt('Conv2d_1a_3x3', 1))
    branch_2 = conv2d_bn(x, 256, 1, name=name_fmt('Conv2d_0a_1x1', 2))
    branch_2 = conv2d_bn(branch_2, 256, 3, name=name_fmt('Conv2d_0b_3x3', 2))
    branch_2 = conv2d_bn(branch_2,
                         256,
                         3,
                         strides=2,
                         padding='valid',
                         name=name_fmt('Conv2d_1a_3x3', 2))
    branch_pool = MaxPooling2D(3,
                               strides=2,
                               padding='valid',
                               name=name_fmt('MaxPool_1a_3x3', 3))(x)
    branches = [branch_0, branch_1, branch_2, branch_pool]
    x = Concatenate(axis=channel_axis, name='Mixed_7a')(branches)

    # 5x Block8 (Inception-ResNet-C block):
    for block_idx in range(1, 6):
        x = _inception_resnet_block(x,
                                    scale=0.2,
                                    block_type='Block8',
                                    block_idx=block_idx)
    x = _inception_resnet_block(x,
                                scale=1.,
                                activation=None,
                                block_type='Block8',
                                block_idx=6)

    # Classification block
    x = GlobalAveragePooling2D(name='AvgPool')(x)
    x = Dropout(1.0 - dropout_keep_prob, name='Dropout')(x)
    # Bottleneck
    x = Dense(classes, use_bias=False, name='Bottleneck')(x)
    bn_name = _generate_layer_name('BatchNorm', prefix='Bottleneck')
    x = BatchNormalization(momentum=0.995, epsilon=0.001, scale=False,
                           name=bn_name)(x)

    # Create model
    model = Model(inputs, x, name='inception_resnet_v1')
    if weights_path is not None:
        model.load_weights(weights_path)

    return model

In [27]:
facenet_model = InceptionResNetV1(input_shape=(160, 160, 3),classes=128)

#### 2) 모델에 가중치 적용

In [28]:
## FaceNet 가중치 파일 경로 설정
weights_path = '/content/drive/MyDrive/project4/Datasets/Keras/facenet_model_weights.npz'
weights_path

'/content/drive/MyDrive/project4/Datasets/Keras/facenet_model_weights.npz'

In [29]:
## 가중치 파일 불러오기
loaded_weights = np.load(weights_path)

loaded_weights

NpzFile '/content/drive/MyDrive/project4/Datasets/Keras/facenet_model_weights.npz' with keys: arr_0, arr_1, arr_2, arr_3, arr_4...

In [30]:
## FaceNet 각 레이어에 가중치 적용
facenet_model.set_weights([loaded_weights[key] for key in loaded_weights])

#### 3) 모델의 가중치 업데이트 방지 (선택사항)

In [31]:
## FaceNet 전체 레이어 확인
facenet_model.layers

[<InputLayer name=input_layer_2, built=True>,
 <Conv2D name=Conv2d_1a_3x3, built=True>,
 <BatchNormalization name=Conv2d_1a_3x3_BatchNorm, built=True>,
 <Activation name=Conv2d_1a_3x3_Activation, built=True>,
 <Conv2D name=Conv2d_2a_3x3, built=True>,
 <BatchNormalization name=Conv2d_2a_3x3_BatchNorm, built=True>,
 <Activation name=Conv2d_2a_3x3_Activation, built=True>,
 <Conv2D name=Conv2d_2b_3x3, built=True>,
 <BatchNormalization name=Conv2d_2b_3x3_BatchNorm, built=True>,
 <Activation name=Conv2d_2b_3x3_Activation, built=True>,
 <MaxPooling2D name=MaxPool_3a_3x3, built=True>,
 <Conv2D name=Conv2d_3b_1x1, built=True>,
 <BatchNormalization name=Conv2d_3b_1x1_BatchNorm, built=True>,
 <Activation name=Conv2d_3b_1x1_Activation, built=True>,
 <Conv2D name=Conv2d_4a_3x3, built=True>,
 <BatchNormalization name=Conv2d_4a_3x3_BatchNorm, built=True>,
 <Activation name=Conv2d_4a_3x3_Activation, built=True>,
 <Conv2D name=Conv2d_4b_3x3, built=True>,
 <BatchNormalization name=Conv2d_4b_3x3_BatchNor

In [32]:
## FaceNet 전체 레이어 가중치 업데이트 방지
for l in facenet_model.layers :
    l.trainable = False

### (2) 모델 구조 변형

* **세부 요구사항**
    - 우리의 문제에 맞게 모델을 변형해야 합니다.
    - 예시 코드에서 사용한 라이브러리
        - keras

#### 1) 추가 모델링

In [33]:
ol=Dense(1,activation='sigmoid')(facenet_model.output)

In [34]:
facenet_model=Model(inputs=facenet_model.input,outputs=ol)

In [35]:
facenet_model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 160, 160, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv2d_1a_3x3 (Conv2D)    │ (None, 79, 79, 32)     │            864 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv2d_1a_3x3_BatchNorm   │ (None, 79, 79, 32)     │             96 │ Conv2d_1a_3x3[0][0]    │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv2d_1a_3x3_Activation  │ (None, 79, 79, 32)     │              0 │ Conv2d_1a_3x3_BatchNo… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv2d_2a_3x3 (Conv2D)    │ (None, 77, 77, 32)     │          9,216 │ Conv2d_1a_3x3_Activat… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv2d_2a_3x3_BatchNorm   │ (None, 77, 77, 32)     │             96 │ Conv2d_2a_3x3[0][0]    │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv2d_2a_3x3_Activation  │ (None, 77, 77, 32)     │              0 │ Conv2d_2a_3x3_BatchNo… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv2d_2b_3x3 (Conv2D)    │ (None, 77, 77, 64)     │         18,432 │ Conv2d_2a_3x3_Activat… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv2d_2b_3x3_BatchNorm   │ (None, 77, 77, 64)     │            192 │ Conv2d_2b_3x3[0][0]    │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv2d_2b_3x3_Activation  │ (None, 77, 77, 64)     │              0 │ Conv2d_2b_3x3_BatchNo… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ MaxPool_3a_3x3            │ (None, 38, 38, 64)     │              0 │ Conv2d_2b_3x3_Activat… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv2d_3b_1x1 (Conv2D)    │ (None, 38, 38, 80)     │          5,120 │ MaxPool_3a_3x3[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv2d_3b_1x1_BatchNorm   │ (None, 38, 38, 80)     │            240 │ Conv2d_3b_1x1[0][0]    │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv2d_3b_1x1_Activation  │ (None, 38, 38, 80)     │              0 │ Conv2d_3b_1x1_BatchNo… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv2d_4a_3x3 (Conv2D

 Total params: 22,824,656 (87.07 MB)

 Trainable params: 16,512 (64.50 KB)

 Non-trainable params: 22,808,144 (87.01 MB)

## 5.미션3

학습된 모델로 추론하여 성능 지표를 확인하고 모델을 개선시키세요.

그 후, 모델의 구조와 가중치를 **반드시 저장**하여 여러분의 노트북에 옮기세요.

- 1) 다양한 모델을 사용해보세요.
    - 모델에 정해진 정답은 없습니다.
    - 성능 지표에서 무엇이 중요한지 깊게 생각하세요.
    - 사전 학습된 FaceNet 모델을 사용하셔도 좋고, 아예 독창적으로 여러분만의 모델을 만드셔도 좋습니다!
- 2) 모델을 **반드시 저장**하세요.
    - .keras 형태로 우선 Colab에 저장하세요.
    - Colab에 생성된 .keras 파일을 **로컬에 다운로드** 합니다.

### (1) 모델 학습

* **세부 요구사항**
    - 모델 구조를 잘 변형하였다면, 학습도 진행해야 합니다.
        - Keras에서 지원하는 다양한 함수를 사용하세요.
    - 예시 코드에서 사용한 라이브러리
        - keras

#### 1) 학습에 유용한 함수 불러오기

In [ ]:
facenet_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
es=EarlyStopping(patience=3,
                 verbose=1,
                 restore_best_weights=True)

#### 2) 모델 학습

### (2) 모델 추론

* **세부 요구사항**
    - 학습된 모델의 성능을 확인해보세요.
        - 임계값 조절, 클래스 가중치 부여 등으로 모델의 성능을 높여보세요.
    - 예시 코드에서 사용한 라이브러리
        - keras, sklearn

#### 1) 모델 추론

#### 2) 성능 확인

### (3) 모델 저장

* **세부 요구사항**
    - **반드시 반드시 모델을 저장하고 로컬에 다운로드하세요.**
    - 예시 코드에서 사용한 라이브러리
        - keras

#### 1) 모델 저장

In [ ]:
## .keras로 저장해야 안전
.save('')

#### 2) 저장된 모델 체크

In [ ]:
## Colab에 저장된 모델을 불러와 확인
temp_model = keras.saving.load_model('')
temp_model.summary()